## Session 1: Advanced Monte Carlo Techniques (90 min)

### Exercise 1.1: Understanding Parameter Distributions

In [ ]:
# Setup
import sys
sys.path.insert(0, '../../..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("✅ Environment ready for advanced Monte Carlo analysis")

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Exercise 1.1a: Explore different distribution types

# PoliSim uses these distributions for different economic variables
distributions = {
    'GDP Growth': stats.norm(loc=0.022, scale=0.008),       # Normal: symmetric
    'Inflation': stats.norm(loc=0.025, scale=0.005),        # Normal: symmetric
    'Healthcare Inflation': stats.lognorm(s=0.2, scale=0.04),  # Lognormal: right-skewed
    'Recession Prob': stats.beta(a=2, b=8),                  # Beta: bounded [0,1]
    'Interest Rate Shock': stats.t(df=5, scale=0.005)        # t-dist: fat tails
}

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, (name, dist) in zip(axes, distributions.items()):
    samples = dist.rvs(10000)
    ax.hist(samples, bins=50, density=True, alpha=0.7, color='steelblue')
    
    # Overlay theoretical PDF
    x = np.linspace(dist.ppf(0.001), dist.ppf(0.999), 100)
    ax.plot(x, dist.pdf(x), 'r-', linewidth=2, label='PDF')
    
    ax.set_title(f'{name}\nMean={samples.mean():.4f}, Std={samples.std():.4f}')
    ax.grid(True, alpha=0.3)

# Hide extra subplot
axes[-1].axis('off')

plt.tight_layout()
plt.suptitle('Economic Parameter Distributions in PoliSim', y=1.02, fontsize=14)
plt.show()

### Exercise 1.2: Correlation Structure

In [ ]:
# Exercise 1.2a: Model correlated economic shocks

# Correlation matrix for economic variables
correlation_matrix = np.array([
    [1.00, -0.30,  0.20, -0.60,  0.40],  # GDP Growth
    [-0.30, 1.00,  0.60,  0.30, -0.20],  # Inflation
    [0.20,  0.60,  1.00,  0.10,  0.10],  # Healthcare Inflation
    [-0.60, 0.30,  0.10,  1.00, -0.50],  # Unemployment
    [0.40, -0.20,  0.10, -0.50,  1.00]   # Interest Rate
])

variable_names = ['GDP Growth', 'Inflation', 'Healthcare Inflation', 'Unemployment', 'Interest Rate']

# Visualize correlation matrix
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)

ax.set_xticks(range(len(variable_names)))
ax.set_yticks(range(len(variable_names)))
ax.set_xticklabels(variable_names, rotation=45, ha='right')
ax.set_yticklabels(variable_names)

# Add correlation values
for i in range(len(variable_names)):
    for j in range(len(variable_names)):
        text = ax.text(j, i, f'{correlation_matrix[i, j]:.2f}',
                       ha='center', va='center', color='black', fontsize=10)

plt.colorbar(im, label='Correlation')
plt.title('Economic Variable Correlation Structure')
plt.tight_layout()
plt.show()

In [ ]:
# Exercise 1.2b: Generate correlated random variables using Cholesky decomposition

def generate_correlated_samples(n_samples, correlation_matrix, means, stds):
    """Generate correlated normal samples using Cholesky decomposition."""
    n_vars = len(means)
    
    # Cholesky decomposition
    L = np.linalg.cholesky(correlation_matrix)
    
    # Generate independent standard normal samples
    independent_samples = np.random.standard_normal((n_samples, n_vars))
    
    # Transform to correlated samples
    correlated_samples = independent_samples @ L.T
    
    # Scale and shift
    correlated_samples = correlated_samples * stds + means
    
    return correlated_samples

# Generate samples
means = np.array([0.022, 0.025, 0.04, 0.05, 0.04])
stds = np.array([0.008, 0.005, 0.01, 0.01, 0.008])

samples = generate_correlated_samples(10000, correlation_matrix, means, stds)
samples_df = pd.DataFrame(samples, columns=variable_names)

print("Generated Correlated Samples:")
print(f"Sample correlation matrix:\n{samples_df.corr().round(2)}")
print(f"\nTarget correlation matrix:\n{pd.DataFrame(correlation_matrix, columns=variable_names, index=variable_names).round(2)}")

### Exercise 1.3: Convergence Analysis

In [ ]:
# Exercise 1.3: Analyze Monte Carlo convergence

def simulate_debt_trajectory(n_iterations, years=10):
    """Simplified debt trajectory simulation."""
    base_debt = 34.0  # trillion, 2024
    base_gdp = 28.0   # trillion
    
    trajectories = []
    
    for _ in range(n_iterations):
        debt = base_debt
        gdp = base_gdp
        
        for year in range(years):
            gdp_growth = np.random.normal(0.022, 0.008)
            deficit_rate = np.random.normal(0.06, 0.02)  # deficit as % of GDP
            
            gdp *= (1 + gdp_growth)
            debt += gdp * deficit_rate
        
        trajectories.append(debt / gdp * 100)  # Debt-to-GDP ratio
    
    return np.array(trajectories)

# Test convergence at different iteration counts
iteration_counts = [100, 500, 1000, 5000, 10000, 50000]
convergence_data = []

for n in iteration_counts:
    results = simulate_debt_trajectory(n)
    convergence_data.append({
        'iterations': n,
        'mean': np.mean(results),
        'std': np.std(results),
        'p5': np.percentile(results, 5),
        'p95': np.percentile(results, 95)
    })

conv_df = pd.DataFrame(convergence_data)
print("Convergence Analysis:")
print(conv_df.to_string(index=False))

In [ ]:
# Visualize convergence
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean convergence
ax1 = axes[0]
ax1.plot(conv_df['iterations'], conv_df['mean'], 'b-o', markersize=8)
ax1.axhline(conv_df.iloc[-1]['mean'], color='r', linestyle='--', alpha=0.5, label='Final Mean')
ax1.fill_between(conv_df['iterations'], 
                 conv_df['mean'] - conv_df['std']/np.sqrt(conv_df['iterations']),
                 conv_df['mean'] + conv_df['std']/np.sqrt(conv_df['iterations']),
                 alpha=0.3)
ax1.set_xlabel('Number of Iterations')
ax1.set_ylabel('Mean Debt-to-GDP Ratio (%)')
ax1.set_title('Mean Convergence')
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Confidence interval convergence
ax2 = axes[1]
ax2.fill_between(conv_df['iterations'], conv_df['p5'], conv_df['p95'], alpha=0.3, label='90% CI')
ax2.plot(conv_df['iterations'], conv_df['p5'], 'g-', linewidth=2)
ax2.plot(conv_df['iterations'], conv_df['p95'], 'g-', linewidth=2)
ax2.set_xlabel('Number of Iterations')
ax2.set_ylabel('Debt-to-GDP Ratio (%)')
ax2.set_title('90% Confidence Interval Convergence')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

### 📝 Analysis Questions - Session 1

1. Why is the lognormal distribution used for healthcare inflation?
2. How does correlation affect the range of outcomes?
3. At what iteration count does the mean stabilize to within 0.1%?
4. Why do confidence intervals converge slower than the mean?

---

## Session 2: Custom Policy Design (90 min)

### Exercise 2.1: Policy Builder Framework

In [ ]:
# Exercise 2.1a: Define policy structure
try:
    from core.policy_builder import PolicyBuilder
    print("✅ PolicyBuilder loaded from core")
except ImportError:
    print("Creating custom PolicyBuilder class")

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Callable
from enum import Enum

class PolicyDomain(Enum):
    REVENUE = "revenue"
    HEALTHCARE = "healthcare"
    SOCIAL_SECURITY = "social_security"
    DEFENSE = "defense"
    DISCRETIONARY = "discretionary"
    INTEREST = "interest"

@dataclass
class PolicyComponent:
    """A single policy provision."""
    name: str
    domain: PolicyDomain
    description: str
    parameters: Dict[str, float]
    start_year: int = 2025
    end_year: Optional[int] = None
    phase_in_years: int = 0
    confidence: float = 0.8  # Estimation confidence
    
    def calculate_impact(self, year: int, baseline: Dict) -> Dict[str, float]:
        """Calculate fiscal impact for a given year."""
        if year < self.start_year:
            return {'revenue': 0, 'spending': 0}
        
        # Phase-in factor
        if self.phase_in_years > 0:
            years_active = year - self.start_year + 1
            phase_factor = min(1.0, years_active / self.phase_in_years)
        else:
            phase_factor = 1.0
        
        # Calculate based on domain
        impact = self._domain_impact(year, baseline, phase_factor)
        return impact
    
    def _domain_impact(self, year, baseline, phase_factor):
        """Calculate domain-specific impact."""
        # Override in subclasses or use parameter-based calculation
        return {'revenue': 0, 'spending': 0}

@dataclass
class PolicyPackage:
    """A collection of policy components."""
    name: str
    description: str
    components: List[PolicyComponent] = field(default_factory=list)
    
    def add_component(self, component: PolicyComponent):
        self.components.append(component)
    
    def total_impact(self, year: int, baseline: Dict) -> Dict[str, float]:
        """Calculate total impact of all components."""
        total = {'revenue': 0, 'spending': 0}
        for component in self.components:
            impact = component.calculate_impact(year, baseline)
            total['revenue'] += impact.get('revenue', 0)
            total['spending'] += impact.get('spending', 0)
        return total

print("\nPolicy framework classes defined:")
print("  - PolicyDomain (enum)")
print("  - PolicyComponent (dataclass)")
print("  - PolicyPackage (dataclass)")

In [ ]:
# Exercise 2.1b: Create specific policy components

class TaxRateChange(PolicyComponent):
    """Policy component for tax rate changes."""
    
    def _domain_impact(self, year, baseline, phase_factor):
        base_revenue = baseline.get('total_revenue', 4500)  # billions
        rate_change = self.parameters.get('rate_change', 0)
        elasticity = self.parameters.get('elasticity', 0.9)  # Behavioral response
        
        # Revenue impact with behavioral response
        revenue_change = base_revenue * rate_change * elasticity * phase_factor
        return {'revenue': revenue_change, 'spending': 0}

class SpendingCut(PolicyComponent):
    """Policy component for spending reductions."""
    
    def _domain_impact(self, year, baseline, phase_factor):
        category = self.parameters.get('category', 'discretionary')
        base_spending = baseline.get(f'{category}_spending', 1000)
        cut_percentage = self.parameters.get('cut_percentage', 0)
        
        spending_reduction = base_spending * cut_percentage * phase_factor
        return {'revenue': 0, 'spending': -spending_reduction}

class HealthcareReform(PolicyComponent):
    """Policy component for healthcare system changes."""
    
    def _domain_impact(self, year, baseline, phase_factor):
        federal_change = self.parameters.get('federal_cost_change', 0)
        admin_savings = self.parameters.get('admin_savings', 0)
        
        base_federal = baseline.get('healthcare_federal', 1500)
        
        # Federal spending change (could increase or decrease)
        spending_change = base_federal * federal_change * phase_factor
        
        # Admin savings reduce spending
        admin_reduction = base_federal * admin_savings * phase_factor
        
        total_spending_change = spending_change - admin_reduction
        return {'revenue': 0, 'spending': total_spending_change}

print("Specialized policy component classes:")
print("  - TaxRateChange")
print("  - SpendingCut")
print("  - HealthcareReform")

In [ ]:
# Exercise 2.1c: Build a comprehensive policy package

# Create policy package
balanced_reform = PolicyPackage(
    name="Balanced Fiscal Reform 2025",
    description="A comprehensive package addressing revenue, healthcare, and discretionary spending"
)

# Add tax reform
top_rate_increase = TaxRateChange(
    name="Top Rate Increase",
    domain=PolicyDomain.REVENUE,
    description="Increase top marginal rate from 37% to 39.6%",
    parameters={'rate_change': 0.02, 'elasticity': 0.85},
    start_year=2025,
    phase_in_years=1
)
balanced_reform.add_component(top_rate_increase)

# Add healthcare reform
usgha_transition = HealthcareReform(
    name="USGHA Transition",
    domain=PolicyDomain.HEALTHCARE,
    description="Transition to US Global Health Assurance system",
    parameters={'federal_cost_change': 0.30, 'admin_savings': 0.08},
    start_year=2026,
    phase_in_years=5,
    confidence=0.7
)
balanced_reform.add_component(usgha_transition)

# Add discretionary cut
efficiency_savings = SpendingCut(
    name="Federal Efficiency Initiative",
    domain=PolicyDomain.DISCRETIONARY,
    description="5% reduction in non-defense discretionary through efficiency",
    parameters={'category': 'discretionary', 'cut_percentage': 0.05},
    start_year=2025,
    phase_in_years=3
)
balanced_reform.add_component(efficiency_savings)

print(f"Policy Package: {balanced_reform.name}")
print(f"Description: {balanced_reform.description}")
print(f"\nComponents ({len(balanced_reform.components)}):")
for comp in balanced_reform.components:
    print(f"  - {comp.name} ({comp.domain.value}): {comp.description}")

### Exercise 2.2: Simulate Custom Policy

In [ ]:
# Exercise 2.2: Project policy package impact

def project_policy_impact(policy_package: PolicyPackage, years: range):
    """Project 10-year fiscal impact of policy package."""
    baseline = {
        'total_revenue': 4500,
        'discretionary_spending': 1600,
        'healthcare_federal': 1500,
        'total_spending': 6800
    }
    
    projections = []
    cumulative_revenue = 0
    cumulative_spending = 0
    
    for year in years:
        impact = policy_package.total_impact(year, baseline)
        cumulative_revenue += impact['revenue']
        cumulative_spending += impact['spending']
        
        projections.append({
            'year': year,
            'annual_revenue': impact['revenue'],
            'annual_spending': impact['spending'],
            'annual_net': impact['revenue'] - impact['spending'],
            'cumulative_net': cumulative_revenue - cumulative_spending
        })
        
        # Update baseline for next year (simplified growth)
        baseline['total_revenue'] *= 1.035
        baseline['discretionary_spending'] *= 1.02
        baseline['healthcare_federal'] *= 1.06
        baseline['total_spending'] *= 1.04
    
    return pd.DataFrame(projections)

# Project our policy package
years = range(2024, 2035)
impact_df = project_policy_impact(balanced_reform, years)

print("10-Year Policy Impact Projection ($ Billions):")
print(impact_df.to_string(index=False))

In [ ]:
# Visualize policy impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Annual impact
ax1 = axes[0]
ax1.bar(impact_df['year'], impact_df['annual_revenue'], label='Revenue Change', alpha=0.7, color='green')
ax1.bar(impact_df['year'], impact_df['annual_spending'], label='Spending Change', alpha=0.7, color='red')
ax1.plot(impact_df['year'], impact_df['annual_net'], 'k--', linewidth=2, marker='o', label='Net Effect')
ax1.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax1.set_xlabel('Year')
ax1.set_ylabel('Impact ($ Billions)')
ax1.set_title('Annual Fiscal Impact')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cumulative impact
ax2 = axes[1]
ax2.fill_between(impact_df['year'], impact_df['cumulative_net'], 
                 where=impact_df['cumulative_net']>=0, alpha=0.3, color='green', label='Deficit Reduction')
ax2.fill_between(impact_df['year'], impact_df['cumulative_net'], 
                 where=impact_df['cumulative_net']<0, alpha=0.3, color='red', label='Deficit Increase')
ax2.plot(impact_df['year'], impact_df['cumulative_net'], 'b-', linewidth=2, marker='o')
ax2.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax2.set_xlabel('Year')
ax2.set_ylabel('Cumulative Impact ($ Billions)')
ax2.set_title('Cumulative Deficit Impact')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle(f'Impact of "{balanced_reform.name}"', y=1.02, fontsize=14)
plt.show()

### 📝 Your Turn: Design Your Own Policy Package

Create a custom policy package with at least 3 components from different domains.

In [ ]:
# YOUR CODE HERE: Create your own policy package

my_policy = PolicyPackage(
    name="My Custom Reform",  # Change this
    description="Describe your policy"  # Change this
)

# Add your components here...
# Example:
# my_policy.add_component(TaxRateChange(...))
# my_policy.add_component(SpendingCut(...))
# my_policy.add_component(HealthcareReform(...))

# Project impact
# my_impact = project_policy_impact(my_policy, years)
# print(my_impact)

---

## Session 3: API Integration & Automation (60 min)

### Exercise 3.1: API Basics

In [ ]:
# Exercise 3.1a: Understanding the API structure
import json

# API endpoint structure
api_endpoints = {
    'simulation': {
        'POST /api/v1/simulate': 'Run a simulation',
        'POST /api/v1/simulate/batch': 'Run multiple simulations',
        'GET /api/v1/simulate/{id}': 'Get simulation results',
        'GET /api/v1/simulate/{id}/status': 'Check simulation status'
    },
    'policy': {
        'GET /api/v1/policies': 'List available policies',
        'GET /api/v1/policies/{id}': 'Get policy details',
        'POST /api/v1/policies': 'Create custom policy',
        'POST /api/v1/policies/compare': 'Compare policies'
    },
    'data': {
        'GET /api/v1/data/baseline': 'Get CBO baseline data',
        'GET /api/v1/data/historical': 'Get historical data',
        'GET /api/v1/data/parameters': 'Get model parameters'
    }
}

print("PoliSim API Endpoints:")
for category, endpoints in api_endpoints.items():
    print(f"\n{category.upper()}")
    for endpoint, description in endpoints.items():
        print(f"  {endpoint}: {description}")

In [ ]:
# Exercise 3.1b: Simulate API requests

def mock_api_request(endpoint: str, method: str = 'GET', data: dict = None):
    """Simulate API request for demonstration."""
    
    # Simulate responses
    if endpoint == '/api/v1/data/baseline':
        return {
            'status': 'success',
            'data': {
                'start_year': 2024,
                'end_year': 2034,
                'revenue_2024': 4520,
                'spending_2024': 6870,
                'debt_2024': 34100
            }
        }
    
    elif endpoint == '/api/v1/simulate' and method == 'POST':
        iterations = data.get('iterations', 1000)
        return {
            'status': 'success',
            'simulation_id': 'sim_abc123',
            'iterations': iterations,
            'estimated_time_seconds': iterations / 100
        }
    
    elif 'simulate' in endpoint and 'status' in endpoint:
        return {
            'status': 'complete',
            'progress': 100,
            'results_available': True
        }
    
    return {'status': 'error', 'message': 'Unknown endpoint'}

# Test API calls
print("API Request: GET /api/v1/data/baseline")
response = mock_api_request('/api/v1/data/baseline')
print(f"Response: {json.dumps(response, indent=2)}")

print("\nAPI Request: POST /api/v1/simulate")
response = mock_api_request('/api/v1/simulate', 'POST', {'iterations': 10000})
print(f"Response: {json.dumps(response, indent=2)}")

### Exercise 3.2: Building a Client Library

In [ ]:
# Exercise 3.2: Create a Python client for the API

class PoliSimClient:
    """Python client for PoliSim API."""
    
    def __init__(self, base_url: str = 'http://localhost:8000', api_key: str = None):
        self.base_url = base_url
        self.api_key = api_key
        self._session = None
    
    def get_baseline(self) -> dict:
        """Retrieve CBO baseline data."""
        return mock_api_request('/api/v1/data/baseline')
    
    def run_simulation(self, 
                       iterations: int = 10000,
                       start_year: int = 2024,
                       end_year: int = 2034,
                       policy_config: dict = None) -> dict:
        """Run a Monte Carlo simulation."""
        payload = {
            'iterations': iterations,
            'start_year': start_year,
            'end_year': end_year,
            'policy_config': policy_config or {}
        }
        return mock_api_request('/api/v1/simulate', 'POST', payload)
    
    def get_simulation_status(self, simulation_id: str) -> dict:
        """Check simulation status."""
        return mock_api_request(f'/api/v1/simulate/{simulation_id}/status')
    
    def compare_policies(self, policies: list) -> dict:
        """Compare multiple policies."""
        return mock_api_request('/api/v1/policies/compare', 'POST', {'policies': policies})

# Demo the client
client = PoliSimClient()

print("Using PoliSimClient:")

# Get baseline
baseline = client.get_baseline()
print(f"\n1. Baseline Data: {baseline['data']}")

# Run simulation
sim_result = client.run_simulation(iterations=50000)
print(f"\n2. Simulation Started: ID={sim_result['simulation_id']}")

# Check status
status = client.get_simulation_status('sim_abc123')
print(f"\n3. Simulation Status: {status['status']}")

---

## Session 4: Sensitivity Analysis & Reporting (60 min)

### Exercise 4.1: Tornado Diagrams

In [ ]:
# Exercise 4.1: Create sensitivity tornado diagram

# Parameters and their impacts on 2034 debt-to-GDP
sensitivity_results = {
    'GDP Growth': {'low': 135, 'base': 126, 'high': 118},
    'Healthcare Inflation': {'low': 122, 'base': 126, 'high': 132},
    'Interest Rate': {'low': 120, 'base': 126, 'high': 134},
    'Social Security COLA': {'low': 124, 'base': 126, 'high': 129},
    'Defense Spending': {'low': 125, 'base': 126, 'high': 128},
    'Non-Defense Discretionary': {'low': 125, 'base': 126, 'high': 127},
}

# Calculate ranges
params = list(sensitivity_results.keys())
base_value = 126

low_impacts = [sensitivity_results[p]['low'] - base_value for p in params]
high_impacts = [sensitivity_results[p]['high'] - base_value for p in params]

# Sort by total range
total_ranges = [abs(h - l) for l, h in zip(low_impacts, high_impacts)]
sorted_indices = np.argsort(total_ranges)[::-1]

params_sorted = [params[i] for i in sorted_indices]
low_sorted = [low_impacts[i] for i in sorted_indices]
high_sorted = [high_impacts[i] for i in sorted_indices]

# Create tornado diagram
fig, ax = plt.subplots(figsize=(12, 6))

y_pos = np.arange(len(params_sorted))

# Plot low impacts (left of center)
ax.barh(y_pos, low_sorted, height=0.6, color='steelblue', alpha=0.7, label='Low Parameter')
# Plot high impacts (right of center)
ax.barh(y_pos, high_sorted, height=0.6, color='coral', alpha=0.7, label='High Parameter')

# Formatting
ax.axvline(x=0, color='black', linewidth=1)
ax.set_yticks(y_pos)
ax.set_yticklabels(params_sorted)
ax.set_xlabel('Change in 2034 Debt-to-GDP Ratio (percentage points)')
ax.set_title('Sensitivity Analysis: What Drives Debt Projections?')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### Exercise 4.2: Generate Professional Report

In [ ]:
# Exercise 4.2: Generate simulation report

def generate_report(simulation_results: dict, policy_name: str = "Baseline") -> str:
    """Generate markdown report from simulation results."""
    
    report = f"""
# PoliSim Fiscal Projection Report

**Policy Analyzed:** {policy_name}  
**Generated:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}  
**Iterations:** 10,000 Monte Carlo simulations

---

## Executive Summary

This analysis projects federal fiscal outcomes from 2024-2034 under {policy_name} assumptions.

### Key Findings

| Metric | 2024 | 2034 (Mean) | 90% CI |
|--------|------|-------------|--------|
| Debt-to-GDP | 98% | 126% | [118%, 135%] |
| Annual Deficit | $2.3T | $3.8T | [$3.2T, $4.5T] |
| Interest Cost | $0.87T | $1.6T | [$1.4T, $1.9T] |

### Risk Assessment

- **High Risk:** Interest costs exceeding 20% of spending by 2032
- **Medium Risk:** Healthcare costs growing faster than GDP
- **Low Risk:** Social Security trust fund depletion (2033-2035)

---

## Detailed Projections

### Revenue Outlook
Federal revenue is projected to grow at 3.5% annually, reaching $6.2T by 2034.

### Spending Outlook  
Total spending projected to reach $10.0T by 2034, driven by:
- Social Security: $2.5T
- Medicare: $1.5T
- Interest: $1.6T

---

## Methodology

This projection uses PoliSim's Monte Carlo simulation engine with:
- Correlated economic shocks
- CBO-aligned baseline parameters
- Validated against 2024 CBO Budget and Economic Outlook

---

*Report generated by PoliSim v1.0*
"""
    return report

# Generate report
report = generate_report({}, "Balanced Fiscal Reform 2025")
print(report)

---

## 🎯 Full-Day Workshop Completion

**Congratulations!** You've completed the full-day technical workshop.

### Advanced Skills Acquired:
- ✅ Monte Carlo correlation structures
- ✅ Convergence analysis
- ✅ Custom policy design framework
- ✅ API integration patterns
- ✅ Sensitivity analysis (tornado diagrams)
- ✅ Professional report generation

### Next Steps:
1. Apply these techniques to real policy questions
2. Contribute to PoliSim development on GitHub
3. Use the API to build custom applications
4. Share your analysis with the community

---

*Questions? Contribute at [GitHub](https://github.com/GalacticOrgOfDev/polisim)*